In [17]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold


from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG=True
from rdkit.Chem import Descriptors, AllChem, PandasTools

First we read the file moleculus combined, which contains the smiles string and ALDH1_inhibition

In [2]:
filename = 'Molecules_combined.csv'
df = pd.read_csv(filename)
df.head()

,SMILES,ALDH1_inhibition
0,COc1ccccc1CC(NC(C)=O)C(=O)NC1CCN(c2nnnn2-c2ccc...,1
1,O=C(CSc1nc2cccnc2n1Cc1ccccc1)NCc1ccco1,1
2,Cc1cccc2cc(C[NH+](CC3CCCO3)C(c3nnnn3Cc3ccco3)C...,1
3,CCN(CC)c1ccc2c(Cl)c(Br)c(=O)oc2c1,1
4,CS(=O)(=O)N1CCc2cc(-c3csc(NC(=O)Cc4cccs4)n3)ccc21,1


Now we add a new column with RDKit molecules and switch the columns around

In [26]:
PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule')
df = df[['SMILES','Molecule','ALDH1_inhibition']]
df.head()

,SMILES,Molecule,ALDH1_inhibition
0,COc1ccccc1CC(NC(C)=O)C(=O)NC1CCN(c2nnnn2-c2ccc...,<rdkit.Chem.rdchem.Mol object at 0x000002692A4...,1
1,O=C(CSc1nc2cccnc2n1Cc1ccccc1)NCc1ccco1,<rdkit.Chem.rdchem.Mol object at 0x000002692A4...,1
2,Cc1cccc2cc(C[NH+](CC3CCCO3)C(c3nnnn3Cc3ccco3)C...,<rdkit.Chem.rdchem.Mol object at 0x0000026935F...,1
3,CCN(CC)c1ccc2c(Cl)c(Br)c(=O)oc2c1,<rdkit.Chem.rdchem.Mol object at 0x0000026935F...,1
4,CS(=O)(=O)N1CCc2cc(-c3csc(NC(=O)Cc4cccs4)n3)ccc21,<rdkit.Chem.rdchem.Mol object at 0x0000026935F...,1


Check whether all smiles strings could be converted to molecules

In [27]:
#Print the amount of 0 values in the molecule column.
df.Molecule.isna().sum()

0

No 0 values so no need to drop any rows
Now we use the RDKit molecule objects to calculate the morgan fingerprint of each molecule

First we make a list of molecules. Then we turn this into a list of morgan fingerprint (radius =2) bitvectors and store them in a new dataframe

In [33]:
#create list of molecules from dataframe
mol_list = df['Molecule'].tolist()

#Calculate morgan fingerprints 
morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in mol_list]

#Store the bits of the morgan fingerprint in a new dataframe
morgan_fp_list = [list(i) for i in morgan_fp]
fp_df = pd.DataFrame(morgan_fp_list)

#Rename columns
bitnumbers = [i for i in range(1,2049)]
fp_df.columns = bitnumbers
fp_df.head()

,1,2,3,4,5,6,7,8,9,10,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we create a new dataframe combining the smiles strings and the morgan fingerprint bits and save this as a csv file

In [39]:
smiles_column_df = df.iloc[:,0].to_frame()
df_smiles_fp = smiles_column_df.merge(fp_df,left_index=True, right_index=True)
df_smiles_fp.head()
df_smiles_fp.to_csv('desciptors_fingerprints.csv')